In [1]:
from pyspark.sql import SparkSession, functions as F


# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


23/08/25 09:25:26 WARN Utils: Your hostname, Ungs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.17 instead (on interface en0)
23/08/25 09:25:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/25 09:25:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/25 09:25:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/08/25 09:25:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Preprocess the taxi dataset

In [2]:
ydf_jan = spark.read.parquet('../data/landing/2023-01-yellow.parquet')
ydf_jan.printSchema()
ydf_jan.count()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



3066766

In [3]:
gdf_jan = spark.read.parquet('../data/landing/2023-01-green.parquet')
gdf_jan.printSchema()
gdf_jan.count()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



68211

In [4]:
gdf_feb = spark.read.parquet('../data/landing/2023-02-green.parquet')
gdf_feb.printSchema()
gdf_feb.count()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- trip_type: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



64809

In [5]:
ydf_dec = spark.read.parquet('../data/landing/2022-12-yellow.parquet')
ydf_dec.printSchema()
ydf_dec.count()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



3399549

In [6]:
gdf_dec = spark.read.parquet('../data/landing/2022-12-green.parquet')
gdf_dec.printSchema()
gdf_dec.count()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- lpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: integer (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



72439

In [7]:
ydf_apr = spark.read.parquet('../data/landing/2023-04-yellow.parquet')
ydf_apr.printSchema()
ydf_apr.count()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



3288250

In [8]:
sdf_may = spark.read.parquet('../data/landing/2023-05-yellow.parquet')
sdf_may.printSchema()
sdf_may.count()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



3513649

In [9]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in gdf_feb.columns]
gdf_feb = gdf_feb.select(*consistent_col_casing)

# this will be used in the cell below when reading in
gdf_schema = gdf_feb.schema
ydf_schema = ydf_apr.schema

In [10]:
def preprocess(sdf, month, year, colour):
    """
    Make the features in the dataset has the same data types and
    remove some irrelevant features.

    Parameters:
    - sdf: Pyspark dataframe ready to be preprocessed
    - month: Month of the dataset being collected.
    - year: Year of the dataset being collected.
    - colour: Colour of the taxi dataset
    """

    if colour == 'yellow':
        required_cols = ('trip_distance', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'pulocationid', 'dolocationid', 'fare_amount', 'passenger_count')
        sdf_schema = ydf_schema
    else:
        required_cols = ('trip_distance', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
        'pulocationid', 'dolocationid', 'fare_amount', 'passenger_count')  
        sdf_schema = gdf_schema  
    sdf = sdf \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf.columns)])
    
    sdf = sdf.select(*required_cols)

    # output the preprocessed dataset to the raw folder
    sdf \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet(f'../data/raw/{year}-{str(month).zfill(2)}-{colour}')

In [11]:
# preprocess all the dataset from nov 2022 to may 2023
for month in range(1, 6):
    for colour in ['yellow', 'green']:
        input_path = f'../data/landing/2023-{str(month).zfill(2)}-{colour}.parquet'

        sdf = spark.read.parquet(input_path)
        preprocess(sdf, month, 2023, colour)

for month in [11,12]:
    for colour in ['yellow', 'green']:
        input_path = f'../data/landing/2022-{month}-{colour}.parquet'

        sdf = spark.read.parquet(input_path)
        preprocess(sdf, month, 2022, colour)
    

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 1]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/mast30034_env/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/envs/mast30034_env/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/envs/mast30034_env/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

# Preprocess the weather dataset

In [12]:
wdf_22 = spark.read.csv("../data/landing/weather_data/weather-2022.csv", header = True)
wdf_22.printSchema()

root
 |-- STATION: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- SOURCE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- REPORT_TYPE: string (nullable = true)
 |-- CALL_SIGN: string (nullable = true)
 |-- QUALITY_CONTROL: string (nullable = true)
 |-- WND: string (nullable = true)
 |-- CIG: string (nullable = true)
 |-- VIS: string (nullable = true)
 |-- TMP: string (nullable = true)
 |-- DEW: string (nullable = true)
 |-- SLP: string (nullable = true)
 |-- AA1: string (nullable = true)
 |-- AA2: string (nullable = true)
 |-- AA3: string (nullable = true)
 |-- AB1: string (nullable = true)
 |-- AD1: string (nullable = true)
 |-- AE1: string (nullable = true)
 |-- AH1: string (nullable = true)
 |-- AH2: string (nullable = true)
 |-- AH3: string (nullable = true)
 |-- AH4: string (nullable = true)
 |-- AH5: string (nullable = true

In [13]:
wdf_23 = spark.read.csv("../data/landing/weather_data/weather-2023.csv", header = True)
wdf_23.printSchema()

root
 |-- STATION: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- SOURCE: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- REPORT_TYPE: string (nullable = true)
 |-- CALL_SIGN: string (nullable = true)
 |-- QUALITY_CONTROL: string (nullable = true)
 |-- WND: string (nullable = true)
 |-- CIG: string (nullable = true)
 |-- VIS: string (nullable = true)
 |-- TMP: string (nullable = true)
 |-- DEW: string (nullable = true)
 |-- SLP: string (nullable = true)
 |-- AA1: string (nullable = true)
 |-- AA2: string (nullable = true)
 |-- AA3: string (nullable = true)
 |-- AB1: string (nullable = true)
 |-- AD1: string (nullable = true)
 |-- AE1: string (nullable = true)
 |-- AH1: string (nullable = true)
 |-- AH2: string (nullable = true)
 |-- AH3: string (nullable = true)
 |-- AH4: string (nullable = true)
 |-- AH5: string (nullable = true

In [ ]:
wdf_22.show()

23/08/24 01:50:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------------------+------+--------+---------+---------+--------------------+-----------+---------+---------------+--------------+-----------+------------+-------+-------+-------+-----------+-----------+----+---------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------+----+--------------------+------------+------------+------------+----+----+----+----+--------------+----+----+----+----+----+----+----+----+---------+----+----+----+----+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+-------------+-------------+-------------+-------------+-------------+------------------+------------------+------------+---

In [ ]:
wdf_23.show()

+-----------+-------------------+------+--------+---------+---------+--------------------+-----------+---------+---------------+--------------+-----------+------------+-------+-------+-------+-----------+-----------+----+---------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------+----+--------------------+------------+------------+------------+------------+----+----+----+--------------+--------------+----+----+----+----+----+---------+---------+---------+----+-----------+------------------+------------------+------------------+-----------------+-----------------+-----------------+----+--------------------+--------------------+-------------+-------------+-------------+-------------+-------------+------------------+----------

In [14]:
# create a directory to store the weather data
import os
output_dir = f"../data/raw/weathers"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [15]:
# ensure everything has consistent casing
const_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in wdf_22.columns]
wdf_22 = wdf_22.select(*const_col_casing)
const_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in wdf_23.columns]
wdf_23 = wdf_23.select(*const_col_casing)

In [16]:
print(len(wdf_23.columns))

98


In [17]:
print(len(wdf_22.columns))

100


In [ ]:
# retain the relevant features only and output it
required_cols = ("date", "wnd", "cig", "vis", "tmp", "dew", "slp")
wdf_22 = wdf_22.select(*required_cols)
wdf_22.write.mode('overwrite').parquet('../data/raw/weathers/weather-2022')


In [ ]:
# retain the relevant features only and output it
wdf_23 = wdf_23.select(*required_cols)
wdf_23.write.mode('overwrite').parquet('../data/raw/weathers/weather-2023')